In [14]:
import pandas as pd
import re
import country_converter as coco
import csv

# SETUP & DATA ACQUISITION


# Load core Land Matrix datasets
# Standardizing on the semicolon delimiter for LSLA exports
deals = pd.read_csv('deals.csv', sep=';', low_memory=False)
contracts = pd.read_csv('contracts.csv', sep=';', low_memory=False)
locations = pd.read_csv('locations.csv', sep=';', low_memory=False)
investors = pd.read_csv('investors.csv', sep=';', low_memory=False)

# 1. DATA INTEGRATION (MERGING)


# Prepare investor attributes with prefixes to maintain provenance
investors_df = investors.add_prefix('Investor_')

# Construct master relational dataset using left joins
merged_df = deals.merge(contracts[['Deal ID', 'Contract date']], on='Deal ID', how='left')
merged_df = merged_df.merge(locations[['Deal ID', 'Spatial accuracy level', 'Point']], on='Deal ID', how='left')
merged_df = merged_df.merge(
    investors_df[['Investor_Investor ID', 'Investor_Country of registration/origin']], 
    left_on='Operating company: Investor ID', 
    right_on='Investor_Investor ID', 
    how='left'
)

# Feature selection: Filtering for relevant analytical variables
target_features = [
    'Deal ID', 'Target country', 'Deal size', 'Deal scope', 'Intention of investment',
    'Current negotiation status', 'Current implementation status', 'Created at',
    'Presence of land conflicts', 'Displacement of people', 'Spatial accuracy level',
    'Point', 'Nature of the deal', 'Carbon offset project',
    'Operating company: Country of registration/origin',
    'Investor_Country of registration/origin'
]

df = merged_df[target_features].copy()


# 2. SECTOR HARMONIZATION (TAXONOMY MAPPING)

TRUNCATION_MAP = {
    "Oth": "Other", "Fodd": "Fodder", "Conservatio": "Conservation",
    "Land speculatio": "Land speculation", "Oil / Gas extractio": "Oil / Gas extraction"
}

SECTOR_TAXONOMY = {
    "Food crops": "Food crops", "Livestock": "Livestock", "Mining": "Mining",
    "Non-food agricultural commodities": "Non-food agricultural commodities",
    "Biomass for biofuels": "Biomass for biofuels", "Fodder": "Fodder",
    "Oil / Gas extraction": "Oil / Gas extraction", 
    "Forest logging / management": "Forestry Management",
    "Timber plantation": "Timber Plantation", "Wind farm": "Wind Farm",
    "Solar park": "Solar Park", "Renewable energy": "Renewable Energy",
    "For carbon sequestration/REDD": "Carbon Sequestration / REDD",
    "Industry": "Industry", "Tourism": "Tourism", "Land speculation": "Land Speculation",
    "Agriculture unspecified": "Agriculture Unspecified", "Conservation": "Conservation"
}

def clean_sectors(raw_val):
    if pd.isna(raw_val): return ""
    s = str(raw_val).strip()
    
    # Strip metadata artifacts (| and #)
    s = re.sub(r"\|[0-9#.\-a-zA-Z]*", "", s)
    if "#" in s:
        s = re.sub(r"^.*#(?!.*#)", "", s).lstrip("#")
    
    # Correct truncations and map to taxonomy
    for bad, good in TRUNCATION_MAP.items():
        s = s.replace(bad, good)
    
    parts = re.split(r"\s*,\s*", s)
    matches = []
    for p in parts:
        for key, val in SECTOR_TAXONOMY.items():
            if key.lower() in p.lower():
                matches.append(val)
    
    return "; ".join(sorted(set(matches)))

df['sectors_cleaned'] = df['Intention of investment'].apply(clean_sectors)

# 3. GEOSPATIAL HARMONIZATION (ISO3 CODES)

cc = coco.CountryConverter()
ISO_CACHE = {}

def get_iso3_codes(country_string):
    if pd.isna(country_string): return None
    
    # Pre-processing: handle multi-country entries and null artifacts
    s = str(country_string).strip()
    if s.lower() in ["data not available", "selection deleted", "european union", ""]:
        return None
    
    countries = [c.strip() for c in s.split(";\n") if c.strip()]
    codes = []
    for c in countries:
        if c not in ISO_CACHE:
            res = cc.convert(names=c, to="ISO3", not_found=None)
            ISO_CACHE[c] = res
        if ISO_CACHE[c] and ISO_CACHE[c] != "not found":
            codes.append(ISO_CACHE[c])
            
    return "; ".join(sorted(set(codes))) if codes else None

COUNTRY_COLS = [
    "Target country",
    "Operating company: Country of registration/origin",
    "Investor_Country of registration/origin"
]

for col in COUNTRY_COLS:
    df[f"{col}_iso3"] = df[col].apply(get_iso3_codes)


# DOWNLOADING FILE


df.to_csv('landmatrix_final_analytical_dataset.csv', index=False)

print(f"Pipeline complete. Dataset dimensions: {df.shape}")
print("Final file saved as: landmatrix_final_analytical_dataset.csv")

Pipeline complete. Dataset dimensions: (15019, 20)
Final file saved as: landmatrix_final_analytical_dataset.csv
